In [ ]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sys.path += ['/content/drive/Shareddrive/filtered_data']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%cd /content/drive/Shareddrives/cs145project/filtered_data/stocks

/content/drive/Shareddrives/cs145project/filtered_data/stocks


In [ ]:
def parse_stock_name(filename, lower=True):
  sym = filename.split('-')[0]
  return sym.lower() if lower else sym

for file in os.listdir()[:5]:
  stock_df = pd.read_csv(file)
  print(f"{parse_stock_name(file)} {stock_df['Date'][0]} ${stock_df['Close'][0]:.2f}")

dish 2009-08-11 $18.53
agn 2009-08-13 $34.45
hd 2009-08-10 $27.10
mu 2011-04-19 $10.52
bmy 2009-08-13 $21.86


In [ ]:
import os
import pickle
import stat
import time

pickle_path = '/content/drive/Shareddrives/cs145project/pickles'

names = {
      'sgd': 'sgd_model.pickle',
      'spacy': 'spacy_model.pickle',
      'bert': 'bert_model.pickle',
      'neural': 'nn_model.pickle',
      'data_train': 'data_train.pickle',
      'best_nn_model': 'best_nn_model.pickle',
  }

def pickle_obj(obj, name, overwrite=False):
  """ Cache an object registered in the names dictionary above.

  @obj  The object you want to cache
  @name Key into the names dictionary above.
  @overwrite  Overwrite a previously saved version of the pickle. If false and
              a file exists, user is promted whether to continue. Upon 'y',
              overwrite is set to 'True' and the previos file is overwritten.
  """
  # if a file exists and not overwrite, ask for user input to force overwrite.
  try:
    save_path = os.path.join(pickle_path, names[name])
  except KeyError as ke:
    raise KeyError(f"'{name}' is not a supported object name. Supported objects are:\n"
                   f"{names.keys()}")

  conflict_cond = os.path.exists(save_path) and not overwrite
  if conflict_cond:
    # get user input to continue, and proceed only on 'y'
    print(f"A previous pickle for '{name}' was found.")
    print(f"{save_path}")
    while True:
      _overwrite = input("Would you like to overwrite? (y/n): ").lower()
      if _overwrite not in ['y', 'n']:
        print("Invalid response.")
        continue
      else:
        overwrite = True if _overwrite == 'y' else False
        break

  if overwrite:
    print(f"Overwriting '{save_path}' object")
  elif conflict_cond:
    print(f"Not pickling '{name}' object")

  if overwrite or not conflict_cond:
    with open(save_path, 'wb') as fd:
      print(f"pickling '{name}' model...", end="")
      pickle.dump(obj, fd)
      print(f"done. Saved '{name}' model to {save_path}")

def unpickle_obj(name):
  try:
    load_path = os.path.join(pickle_path, names[name])
  except KeyError as ke:
    raise KeyError(f"'{name}' is not a supported object name. Supported objects are:\n"
                   f"{names.keys()}")
  try:
    with open(load_path, 'rb') as fd:
      return pickle.load(fd)
  except FileNotFoundError as err:
    raise FileNotFoundError(f"No pickled file for '{name}' found.\n"
                            f"{err}")

## Training the headline classifier
Here, I'm going to take the headline sentiments data set we have, and train a text classifier to determine if a headline is positive, negative, or neutral.

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier

import six
import sys
sys.modules['sklearn.externals.six'] = six
from mlxtend.classifier import EnsembleVoteClassifier

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
X_train

1116                 wellness services important future .
3838    equipment made vaahto 's plant hollola finland...
4324    juhani j+ærvi , corporate executive vice presi...
2776    transaction , m-real 30 % metsa-botnia upm -- ...
1887         `` pleased welcome tapeks noma cramo group .
                              ...                        
1033    construction scheduled start april-june 2007 c...
3264    gypsii mobile social networking application av...
1653    service intended allow people thirteen mediter...
2607                   company website www.ahlstrom.com .
2732    employees would remain oulu plant support func...
Name: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing ., Length: 3876, dtype: object

In [ ]:
# Model building
model = make_pipeline(TfidfVectorizer(), SGDClassifier())
# Training the model with the training data
model.fit(X_train, y_train)

# Training accuracy
predicted_train = model.predict(X_train)
print(f"train accuracy: {accuracy_score(y_train, predicted_train)}")

# Predicting the test data categories
predicted_categories = model.predict(X_test)
print(f"test accuracy: {accuracy_score(y_test, predicted_categories)}")

train accuracy: 0.9780701754385965
test accuracy: 0.7492260061919505


In [ ]:
# pickle_obj(model, 'sgd')

## TO DO: Classifying Headlines
In this section, we will take our trained classifier and use it to classify headlines of the 50 stocks within our Headlines dataset.

In [ ]:
# Change Directories Again
%cd /content/drive/Shareddrives/cs145project/filtered_data/

/content/drive/Shareddrives/cs145project/filtered_data


In [ ]:
path="top-50-analyst-ratings-processed.csv"
headlines=pd.read_csv(path,encoding = "ISO-8859-1")
headlines['title']=headlines['title'].apply(clean_text)
headlines.head()

,title,date,stock
0,"piper sandler maintains overweight adobe, lowe...",2020-03-31 06:18:00-04:00,ADBE
1,shares several technology companies trading hi...,2020-03-30 10:23:00-04:00,ADBE
2,"shares several technology, semiconductor softw...",2020-03-27 11:30:00-04:00,ADBE
3,"cramer reveals stock favorites, says intuitive...",2020-03-27 10:10:00-04:00,ADBE
4,shares several software companies trading high...,2020-03-26 10:38:00-04:00,ADBE


In [ ]:
input=headlines['title']
cats=model.predict(input)
headlines['sentiment']=cats

from collections import Counter
counts = Counter(headlines['sentiment'])
print("Sentiment Distribution:")
print("    Sentiment\tcount\t% of dataset")
for sentiment, count in counts.items():
  print(f"    {sentiment}\t{count}\t{((count/counts.total())*100):.3f}%")

headlines.head()

Sentiment Distribution:
    Sentiment	count	% of dataset
    neutral	87940	75.167%
    positive	20751	17.737%
    negative	8302	7.096%


,title,date,stock,sentiment
0,"piper sandler maintains overweight adobe, lowe...",2020-03-31 06:18:00-04:00,ADBE,neutral
1,shares several technology companies trading hi...,2020-03-30 10:23:00-04:00,ADBE,positive
2,"shares several technology, semiconductor softw...",2020-03-27 11:30:00-04:00,ADBE,positive
3,"cramer reveals stock favorites, says intuitive...",2020-03-27 10:10:00-04:00,ADBE,neutral
4,shares several software companies trading high...,2020-03-26 10:38:00-04:00,ADBE,neutral


# Transform datasets

In [ ]:
sys.path.extend(['/content/drive/Shareddrives/cs145project/code', '/content/drive/Shareddrives/cs145project'])

%cd /content/drive/Shareddrives/cs145project/split_data

/content/drive/Shareddrives/cs145project/split_data


In [ ]:
majority_pickle_path = '/content/drive/Shareddrives/cs145project/filtered_data/majority_vote_sentiment.pkl'

majority_df = pd.read_pickle(majority_pickle_path)

In [ ]:
def get_predictions(df):
  return majority_df.loc[majority_df['title'].isin(df['title']) & majority_df['stock'].isin(df['stock'])]

In [ ]:
news = "train_news/ADBE-analyst-rating-train.csv"
news_df = pd.read_csv(news)
prediction_test = get_predictions(news_df)["sentiment"]
prediction_test

10       neutral
14      negative
15      negative
16       neutral
17       neutral
          ...   
1969     neutral
1970     neutral
1971     neutral
1972     neutral
1973     neutral
Name: sentiment, Length: 1939, dtype: object

In [ ]:

from collections import Counter

def make_filenames_dict(dir):
  return {parse_stock_name(f).lower(): os.path.join('.', dir, f) for f in os.listdir(os.path.join('.', dir))}

def load_data(filename):
  return pd.read_csv(filename, encoding = "ISO-8859-1")

# Data Engineering

In [ ]:
%cd /content/drive/Shareddrives/cs145project/split_data/

/content/drive/Shareddrives/cs145project/split_data


In [ ]:
# Process SPY DF for Data Engineering
def process_spy(spy_path):
  df = pd.read_csv(spy_path)
  df["spy1daydiff"] = df["Close"].diff(1) / df["Close"].shift(1)
  df["spy2daydiff"] = df["Close"].diff(2) / df["Close"].shift(2)
  df["spy3daydiff"] = df["Close"].diff(3) / df["Close"].shift(3)
  df["spy4daydiff"] = df["Close"].diff(4) / df["Close"].shift(4)
  df["spy5daydiff"] = df["Close"].diff(5) / df["Close"].shift(5)
  df["spy10daydiff"] = df["Close"].diff(10) / df["Close"].shift(10)
  df["spy20daydiff"] = df["Close"].diff(20) / df["Close"].shift(20)
  df["spy30daydiff"] = df["Close"].diff(30) / df["Close"].shift(30)

  df = df.drop(columns=["Open", "High", "Low", "Close", "Adj Close", "Volume"])
  df = df.dropna()
  return df

In [ ]:
from nltk.corpus.reader.mte import xpath

# Combine given stock, news, and spy DFs into one merged DF
def combine_stock_news(stock, news, spy):
  stock = stock
  news = news


  # Drop Label
  stock = stock.drop('label', axis=1)

  # Create label for row a: (x[a+1] - x[a]) / x[a]
  stock["label"] = (stock["Close"].shift(-1) - stock["Close"]) / stock["Close"]

  # Create % price shifts across days
  stock["1daydiff"] = stock["Close"].diff(1) / stock["Close"].shift(1)
  stock["2daydiff"] = stock["Close"].diff(2) / stock["Close"].shift(2)
  stock["3daydiff"] = stock["Close"].diff(3) / stock["Close"].shift(3)
  stock["4daydiff"] = stock["Close"].diff(4) / stock["Close"].shift(4)
  stock["5daydiff"] = stock["Close"].diff(5) / stock["Close"].shift(5)
  stock["10daydiff"] = stock["Close"].diff(10) / stock["Close"].shift(10)
  stock["20daydiff"] = stock["Close"].diff(20) / stock["Close"].shift(20)
  stock["30daydiff"] = stock["Close"].diff(30) / stock["Close"].shift(30)
  stock = stock.dropna()


  # Merge with S&P to create 5 days percent change compared to market

  stock = stock.merge(spy, on="Date", how="left")
  stock["spy1daydiff"] = stock["1daydiff"] - stock["spy1daydiff"]
  stock["spy2daydiff"] = stock["2daydiff"] - stock["spy2daydiff"]
  stock["spy3daydiff"] = stock["3daydiff"] - stock["spy3daydiff"]
  stock["spy4daydiff"] = stock["4daydiff"] - stock["spy4daydiff"]
  stock["spy5daydiff"] = stock["5daydiff"] - stock["spy5daydiff"]
  stock["spy10daydiff"] = stock["10daydiff"] - stock["spy10daydiff"]
  stock["spy20daydiff"] = stock["20daydiff"] - stock["spy20daydiff"]
  stock["spy30daydiff"] = stock["30daydiff"] - stock["spy30daydiff"]


  # Categorize label into +1, 0, -1 based on % change from yesterday
  stock['label'] = stock['label'].apply(lambda x: 1 if x > 0.005 else (-1 if x < -0.005 else 0))

  news['sentiment'] = get_predictions(news)["sentiment"]

  # Group by 'Date' and 'Sentiment', and aggregate counts
  news = news.groupby(['date', 'sentiment']).size().reset_index(name='Count')

  # Pivot the table to have sentiment types as separate columns
  news = news.pivot_table(index='date', columns='sentiment', values='Count', fill_value=0).reset_index()

  # Perform left merge on stock and news based on date
  df_merged = stock.merge(news, left_on="Date", right_on="date", how="left")
  df_merged = df_merged.fillna(0)

  df_merged = df_merged.drop(columns=["date", "Adj Close", "Date"])

  # Drops first 30 + last 1 rows
  return df_merged

In [ ]:
# Process and combine datasets from 50 companies into 1 DF for training
def combine_datasets():

  # Generate file paths
  train_stock_files = make_filenames_dict("train_stock")
  train_news_files = make_filenames_dict("train_news")
  test_stock_files = make_filenames_dict("test_stock")
  test_news_files = make_filenames_dict("test_news")
  SPY_PATH = "SPY.csv"

  df = pd.DataFrame()

  spy = process_spy(SPY_PATH)

  for stock in train_stock_files.keys():
    # Get file paths for stock
    train_stock = pd.read_csv(train_stock_files[stock])
    train_news = pd.read_csv(train_news_files[stock])
    test_stock = pd.read_csv(test_stock_files[stock])
    test_news = pd.read_csv(test_news_files[stock])

    # Merge train and test DFs
    stock_df = pd.concat([train_stock, test_stock], ignore_index=True)
    news_df = pd.concat([train_news, test_news], ignore_index=True)

    # Data engineer for stock and get DF
    df_new = combine_stock_news(stock = stock_df, news = news_df, spy=spy)

    # Merge train and test into main DF
    df = pd.concat([df, df_new], ignore_index=True)
    print(f"Merged stock: {stock} with shape={df_new.shape} made from {train_stock.shape} and {test_stock.shape}")

  # Fill 0 since some dfs are missing sentiment columns and got autofilled to NaN in the concat
  df = df.fillna(0)

  print(f"Finished merged dataset with shape: {df.shape}")

  return df

In [ ]:
df = combine_datasets()
df.head()

Merged stock: azn with shape=(2644, 22) made from (2664, 8) and (11, 8)
Merged stock: pep with shape=(2644, 22) made from (2664, 8) and (11, 8)
Merged stock: tm with shape=(2633, 22) made from (2673, 8) and (11, 8)
Merged stock: mrk with shape=(2659, 22) made from (2679, 8) and (11, 8)
Merged stock: grpn with shape=(2083, 22) made from (2103, 8) and (11, 8)
Merged stock: fcx with shape=(2649, 22) made from (2669, 8) and (11, 8)
Merged stock: atvi with shape=(2639, 22) made from (2659, 8) and (11, 8)
Merged stock: ms with shape=(2529, 22) made from (2559, 8) and (11, 8)
Merged stock: baba with shape=(1202, 22) made from (1222, 8) and (11, 8)
Merged stock: bmy with shape=(2646, 22) made from (2667, 8) and (11, 8)
Merged stock: adbe with shape=(2644, 25) made from (2664, 8) and (11, 8)
Merged stock: dish with shape=(2648, 22) made from (2668, 8) and (11, 8)
Merged stock: gild with shape=(2644, 22) made from (2664, 8) and (11, 8)
Merged stock: nok with shape=(2649, 22) made from (2669, 8) 

,Open,High,Low,Close,Volume,label,1daydiff,2daydiff,3daydiff,4daydiff,...,spy2daydiff,spy3daydiff,spy4daydiff,spy5daydiff,spy10daydiff,spy20daydiff,spy30daydiff,negative,neutral,positive
0,22.705000,22.709999,22.490000,22.584999,3000600.0,0,0.009160,0.028695,0.025193,-0.009647,...,0.013855,0.015765,-0.007952,-0.005482,0.001799,-0.065860,-0.079769,0.0,0.0,0.0
1,22.700001,22.700001,22.315001,22.475000,3815200.0,-1,-0.004870,0.004245,0.023685,0.020200,...,0.011111,0.012771,0.014676,-0.008914,0.005119,-0.077043,-0.088423,0.0,0.0,0.0
2,22.400000,22.405001,21.969999,21.980000,3416000.0,-1,-0.022025,-0.026788,-0.017873,0.001139,...,0.001797,0.013636,0.015308,0.017157,0.005467,-0.073614,-0.076946,0.0,0.0,0.0
3,21.745001,22.025000,21.730000,21.820000,3321400.0,1,-0.007279,-0.029144,-0.033872,-0.025022,...,0.000215,-0.000759,0.011001,0.012616,0.005123,-0.057682,-0.060973,0.0,0.0,0.0
4,22.000000,22.230000,21.920000,22.105000,3525800.0,0,0.013061,0.005687,-0.016463,-0.021253,...,-0.004510,-0.001594,-0.002574,0.009345,-0.002955,-0.046273,-0.055337,0.0,0.0,0.0


# Split DF into train, val, test for training

In [ ]:
# Split combined DF into a 0.6, 0.2, 0.2 train, val, test split
def split_train_val_test(df):

  # Splitting into training and temporary data with stratification and shuffling
  df_train, df_temp = train_test_split(df, test_size=0.4, stratify=df['label'], shuffle=True, random_state=42)

  # Splitting the temporary data into validation and testing data with stratification and shuffling
  df_val, df_test = train_test_split(df_temp, test_size=0.5, stratify=df_temp['label'], shuffle=True, random_state=42)

  # Split into x and labels

  x_train = df_train.drop(columns=["label"])
  y_train = df_train["label"]

  x_val = df_val.drop(columns=["label"])
  y_val = df_val["label"]

  x_test = df_test.drop(columns=["label"])
  y_test = df_test["label"]

  # Printing the shape of each DataFrame to verify the split
  print("Original DataFram shape:", df.shape, "\n")

  print("x_train DataFrame shape:", x_train.shape)
  print("y_train DataFrame shape:", y_train.shape, "\n")

  print("x_val DataFrame shape:", x_val.shape)
  print("y_val DataFrame shape:", y_val.shape, "\n")

  print("x_test DataFrame shape:", x_test.shape)
  print("y_test DataFrame shape:", y_test.shape)




  return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = split_train_val_test(df)

Original DataFram shape: (115768, 25) 

x_train DataFrame shape: (69460, 24)
y_train DataFrame shape: (69460,) 

x_val DataFrame shape: (23154, 24)
y_val DataFrame shape: (23154,) 

x_test DataFrame shape: (23154, 24)
y_test DataFrame shape: (23154,)


# Neural Network Classifier

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
y_train.value_counts(), y_val.value_counts() , y_test.value_counts()

( 1    24995
 -1    23266
  0    21199
 Name: label, dtype: int64,
  1    8332
 -1    7755
  0    7067
 Name: label, dtype: int64,
  1    8332
 -1    7756
  0    7066
 Name: label, dtype: int64)

# Prepare DataFrames for training on Pytorch

In [ ]:
# Check for device (GPU preferred)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create data tensors
x_train_tensor = torch.tensor(x_train.values).float().to(device)
x_val_tensor = torch.tensor(x_val.values).float().to(device)
x_test_tensor = torch.tensor(x_test.values).float().to(device)

# Add +1 to ys to make them [0, 1, 2] instead of [-1, 0, 1] for training
y_train_tensor = (torch.tensor(y_train.values).float() + 1).to(device)
y_val_tensor = (torch.tensor(y_val.values).float() + 1).to(device)
y_test_tensor = (torch.tensor(y_test.values).float() + 1).to(device)

# Create datasets

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)


print(f"Moved DFs to {train_dataset.tensors[0].device} dataset tensors")

Moved DFs to cuda:0 dataset tensors


In [ ]:
x_train_tensor.device

device(type='cuda', index=0)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, hidden_size=20, n_hidden=2,):
        super(NeuralNetwork, self).__init__()

        # Define the layers
        self.input_layer = nn.Linear(24, hidden_size)  # 24 input nodes, hidden_size hidden nodes
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_size, hidden_size) for _ in range(n_hidden)])  # n_hidden  layers
        self.output_layer = nn.Linear(hidden_size, 3)  # hidden_size hidden nodes, 3 output nodes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.input_layer(x))

        for hidden_layer in self.hidden_layers:
            x = self.relu(hidden_layer(x))

        x = self.output_layer(x)
        return x

# Train Model

In [ ]:
# Model Hyper Parameters
batch_size = 69460
num_epochs = 250
report_interval = 25

learning_rates = [1e-4]
label_smoothing = [0.05]
n_hidden = [6]
hidden_size = [48]


# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
def _train_loop(clf, optimizer, loss_fn):
  train_acc_list = []
  val_acc_list = []
  train_loss_list = []
  epoch_list = []

  # Training loop
  for epoch in range(0, num_epochs+1):
      clf.train()
      running_loss = 0

      for batch_data, batch_labels in train_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.type(torch.LongTensor).to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        output = clf(batch_data).to(device)

        # Compute the loss
        loss = loss_fn(output, batch_labels)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        running_loss += loss.item()

      # Report training and validation accuracy every 10 epochs
      if (epoch % report_interval == 0):
        clf.eval()  # Set the model to evaluation mode

        train_c = 0
        train_t = len(train_dataset)

        val_c = 0
        val_t = len(val_dataset)

        with torch.no_grad(): # Turns off grad calculations for better performance


          # Compute Training Accuracy
          for batch_data, batch_labels in train_loader:
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.type(torch.LongTensor).to(device)

            output = clf(batch_data).to(device)

            pred_labels = torch.argmax(output, dim=1).to(device)
            train_c += torch.sum(pred_labels == batch_labels).item()

          # Compute validation accuracy
          for batch_data, batch_labels in val_loader:
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.type(torch.LongTensor).to(device)

            output = clf(batch_data).to(device)

            pred_labels = torch.argmax(output, dim=1).to(device)
            val_c += torch.sum(pred_labels == batch_labels).item()

        train_acc = (train_c / train_t)
        val_acc = (val_c / val_t)
        epoch_loss = running_loss / len(train_dataset)
        train_loss_list.append(epoch_loss)
        epoch_list.append(epoch)

        # Print the accuracies
        print(f"  Epoch {epoch}: train_loss: {round(epoch_loss, 7)} train_acc: { round(train_acc, 5)}, val_acc: {round(val_acc, 5)}")
        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)

  return train_acc_list, val_acc_list, train_loss_list, epoch_list, clf



def train():
  best_model = None
  best_val_acc = 0.0
  best_val_list = []
  best_train_list = []
  best_train_loss = []
  for lr in learning_rates:
    for ls in label_smoothing:
      for nh in n_hidden:
        for hs in hidden_size:
          #  Instantiate a new nn model
          NNmodel_dev = NeuralNetwork(hidden_size=hs, n_hidden=nh).to(device)

          # Define the loss function
          loss_fn = nn.CrossEntropyLoss(label_smoothing=ls).to(device)

          # set optimizer with learning rate
          optimizer = optim.Adam(NNmodel_dev.parameters(), lr=lr)
          for state in optimizer.state.values():
            for k, v in state.items():
                state[k] = v.to(device)

          # Train
          print(f"Model: learn_rate={lr} label_smooth={ls} num_hidden={nh} hidden_size={hs}")
          train_acc_list, val_acc_list, train_loss_list, epoch_list, clf = _train_loop(clf=NNmodel_dev, optimizer=optimizer, loss_fn=loss_fn)
          dic = {"epochs" : epoch_list, "train acc": train_acc_list, "val acc": val_acc_list, "train loss" : train_loss_list}
          df = pd.DataFrame(dic)
          df.to_csv(f"./outputs/lr={lr}_ls={ls}_nh={nh}_hs={hs}.csv", index=False)

          if max(val_acc_list) > best_val_acc:
            best_val_acc = max(val_acc_list)
            best_val_list = val_acc_list
            best_train_list = train_acc_list
            best_train_loss = train_loss_list
            best_model = clf
  try:
    pickle_obj(best_model, "best_nn_model", overwrite=True)
  except:
    pass

  return best_val_acc, best_val_list, best_train_list, best_train_loss, best_model

None


In [ ]:
best_val_acc, best_val_list, best_train_list, best_train_loss, best_model = train()

Model: learn_rate=0.0001 label_smooth=0.05 num_hidden=6 hidden_size=48
  Epoch 0: train_loss: 0.0328378 train_acc: 0.33496, val_acc: 0.33497
  Epoch 25: train_loss: 0.001948 train_acc: 0.35985, val_acc: 0.35985
  Epoch 50: train_loss: 0.000186 train_acc: 0.33536, val_acc: 0.34137
  Epoch 75: train_loss: 0.0001338 train_acc: 0.33501, val_acc: 0.33497
  Epoch 100: train_loss: 2.56e-05 train_acc: 0.30438, val_acc: 0.30401
  Epoch 125: train_loss: 3.96e-05 train_acc: 0.33517, val_acc: 0.33493
  Epoch 150: train_loss: 0.0001083 train_acc: 0.33521, val_acc: 0.33467
  Epoch 175: train_loss: 5.81e-05 train_acc: 0.33593, val_acc: 0.33541
  Epoch 200: train_loss: 0.0001342 train_acc: 0.33514, val_acc: 0.33463
  Epoch 225: train_loss: 0.0002077 train_acc: 0.3052, val_acc: 0.30526
  Epoch 250: train_loss: 0.0001794 train_acc: 0.33496, val_acc: 0.33497
Overwriting '/content/drive/Shareddrives/cs145project/pickles/best_nn_model.pickle' object
pickling 'best_nn_model' model...done. Saved 'best_nn_mod

In [ ]:
# Test Accuracy
output = best_model(x_test_tensor)
pred_labels = torch.argmax(output, dim=1).to(device)
test_acc = (torch.sum(pred_labels == y_test_tensor).item()) / len(x_test_tensor)
print("Test accuracy: ", test_acc)
print("Predicted Labels: ", pred_labels[0:15].tolist())
print("Correct   Labels: ", [int(x) for x in y_test_tensor[0:15].tolist()])

Test accuracy:  0.3369612162045435
Predicted Labels:  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Correct   Labels:  [0, 1, 0, 0, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2]


In [ ]:
import matplotlib.pyplot as plt

print(training_acc)
plt.plot(training_acc)
plt.plot(testing_acc)

In [ ]:
print(NNmodel.parameters())

In [ ]:
# print weights for first and last layer of NN

for param in NNmodel.input_layer.parameters():
  print("Input Layer:")
  if param.requires_grad:
    print(param[0])

for param in NNmodel.output_layer.parameters():
  print("Output Layer:")
  if param.requires_grad:
    print(param[0])

In [ ]:
# print gradient for first and last layer of NN

for param in NNmodel.input_layer.parameters():
  print("Input Layer:")
  if param.requires_grad:
    print(param.grad[0])

for param in NNmodel.output_layer.parameters():
  print("Output Layer:")
  if param.requires_grad:
    print(param.grad[0])